In [34]:
import base64
import requests
import datetime
from secrets import api_key, client_id, client_secret

In [81]:
class SpotifyAPI():
    # Placeholder for attributes    
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret= None
    token_endpoint = 'https://accounts.spotify.com/api/token'
    user_id = '12185809439'
    playlist_endpoint = f'https://api.spotify.com/v1/users/{user_id}/playlists'
    
    
    def __init__(self, api_key, client_id, client_secret, *args, **kwargs):
        # super init - allows it to call any class that its inheriting from        
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        self.api_key = api_key
    
    def get_client_credentials(self):
        '''
        Returns a base64 encoded string
        '''
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id == None or client_secret == None:
            raise Exception("Must set client_id and client_secret")
        client_cred = f"{client_id}:{client_secret}"
        client_cred_b64 = base64.b64encode(client_cred.encode())
        return client_cred_b64.decode()
    
    def get_token_headers(self):
        client_cred_b64 = self.get_client_credentials()
        return {
        "Authorization": f"Basic {client_cred_b64}"
    }
    
    def get_bearer_headers(self):
        api_key = self.api_key
        return {
        "Authorization": f"Bearer {api_key}"
    }
    
    
    def get_token_data(self):
        return {
        "grant_type": "client_credentials"
    }
    
    def auth(self):
        token_endpoint = self.token_endpoint
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        
        r = requests.post(token_endpoint, data=token_data, headers=token_headers)

        if r.status_code not in range(200, 299):
            return False
        
        print(f"Status Code: {r.status_code}")
        now = datetime.datetime.now()
        token_response = r.json()
        access_token = token_response['access_token']
        expires_in = token_response['expires_in'] # in seconds
        expires = now + datetime.timedelta(seconds=expires_in) # use datetime to calculate the time of expiration
        
        self.access_token = access_token 
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        self.token_res = token_response
        return True
    
    def get_my_playlists(self, limit, offset):
        user_id = '12185809439'
        playlist_endpoint = f'https://api.spotify.com/v1/users/{user_id}/playlists'
        headers = self.get_bearer_headers()
        data = urlencode({"limit": limit, 'offset':offset})
        # A URL ready string to get user playlists
        lookup_url = f"{playlist_endpoint}?{data}"
        
        res = requests.get(lookup_url, headers=headers)
        
        if res.status_code not in range(200, 299):
            raise Exception(res.json())
        
        print(res.status_code)
        self.my_playlists = res.json()['items']
        


In [83]:
spotify = SpotifyAPI(api_key, client_id, client_secret)
spotify.get_my_playlists(limit=15, offset=5)
spotify.my_playlists
print(len(spotify.my_playlists))

200
15


## Retrieve User Playlists
The data should be included in the url - use library urllib.
The format of the GET request is as follows - https://developer.spotify.com/console/get-playlists/?user_id=&limit=10&offset=5

In [9]:
# urlencode - wrap this around a data dictionary to create a URL ready string 
from urllib.parse import urlencode

In [33]:
user_id = '12185809439'
endpoint = f'https://api.spotify.com/v1/users/{user_id}/playlists'
data = urlencode({"limit": 10, 'offset':5})


headers = {
    "Authorization": f"Bearer {oauth2}"
}

# A URL ready string to get user playlists
lookup_url = f"{endpoint}?{data}"

r = requests.get(lookup_url, headers=headers)

r.status_code
r.json()

{'href': 'https://api.spotify.com/v1/users/12185809439/playlists?offset=5&limit=10',
 'items': [{'collaborative': False,
   'description': '',
   'external_urls': {'spotify': 'https://open.spotify.com/playlist/6dTegIN5DZwMnp1Q8ghls1'},
   'href': 'https://api.spotify.com/v1/playlists/6dTegIN5DZwMnp1Q8ghls1',
   'id': '6dTegIN5DZwMnp1Q8ghls1',
   'images': [{'height': 640,
     'url': 'https://mosaic.scdn.co/640/ab67616d0000b2732c7b9cc9ec42b1b310e26b1aab67616d0000b27352f0319fc7a7ab70239c1aefab67616d0000b273ccbbc243b809803ac62ad49aab67616d0000b273de1a3a5eaa0c75bb18e7b597',
     'width': 640},
    {'height': 300,
     'url': 'https://mosaic.scdn.co/300/ab67616d0000b2732c7b9cc9ec42b1b310e26b1aab67616d0000b27352f0319fc7a7ab70239c1aefab67616d0000b273ccbbc243b809803ac62ad49aab67616d0000b273de1a3a5eaa0c75bb18e7b597',
     'width': 300},
    {'height': 60,
     'url': 'https://mosaic.scdn.co/60/ab67616d0000b2732c7b9cc9ec42b1b310e26b1aab67616d0000b27352f0319fc7a7ab70239c1aefab67616d0000b273ccbbc